In [1]:
import os
os.environ["MKL_THREADING_LAYER"] = "GNU"  

import matplotlib
#matplotlib.use('Agg')  # No GUI calls

import matplotlib.pyplot as plt

In [2]:
import torch
from torch import nn
import math
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [3]:
batch_size=32

In [4]:
data_output = pd.read_csv("C:/Users/met48/Desktop/TS-Clustering/SimData/epsteinCV_outputs_active.csv", header=None)

scaler = MinMaxScaler()
data_output_scaled = scaler.fit_transform(data_output)
data_output = pd.DataFrame(data_output_scaled)

# --- Load the conditioning input parameters ---
data_input = pd.read_csv("C:/Users/met48/Desktop/TS-Clustering/SimData/epsteinCV_inputs.csv", sep=" ", header=None)

# --- Combine input + output ---
data = pd.concat([data_input, data_output], axis=1)

# --- Sample to 1280 examples if needed ---
data = data.sample(n=20000, random_state=1)

# --- Split into training and validation sets ---
train_data_pd, valid_data = train_test_split(data, test_size=0.2, random_state=42)

# --- Extract tensors for training ---
train_inputs = torch.tensor(train_data_pd.iloc[:, :3].values, dtype=torch.float32)
train_outputs = torch.tensor(train_data_pd.iloc[:, 3:].values, dtype=torch.float32)

valid_inputs = torch.tensor(valid_data.iloc[:, :3].values, dtype=torch.float32)
valid_outputs = torch.tensor(valid_data.iloc[:, 3:].values, dtype=torch.float32)

In [5]:
data

,0,1,2,0,1,2,3,4,5,6,...,242,243,244,245,246,247,248,249,250,251
30239,0.721832,0.106032,0.315410,0.0,0.088904,0.000000,0.000000,0.001378,0.000689,0.000000,...,0.000000,0.000689,0.000689,0.002757,0.002757,0.002068,0.000689,0.000689,0.002068,0.002757
13381,0.926782,0.043385,0.473899,0.0,0.314955,0.543763,0.549276,0.508615,0.462440,0.407994,...,0.013784,0.019297,0.019297,0.021365,0.033081,0.042040,0.026878,0.027567,0.023432,0.025500
29424,0.913186,0.080047,0.554347,0.0,0.068229,0.004824,0.000689,0.000000,0.000689,0.000000,...,0.000000,0.000689,0.000689,0.001378,0.000000,0.001378,0.004135,0.001378,0.000689,0.000689
3624,0.799078,0.043411,0.749577,0.0,0.013784,0.000689,0.002068,0.001378,0.000689,0.000000,...,0.017919,0.010338,0.008270,0.002757,0.000000,0.000689,0.000689,0.001378,0.001378,0.003446
24564,0.508585,0.216760,0.014888,0.0,0.034459,0.000000,0.000000,0.000000,0.000689,0.000000,...,0.001378,0.000000,0.000689,0.000000,0.000000,0.000689,0.000000,0.001378,0.000689,0.000689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14786,0.961642,0.029016,0.278000,0.0,0.661613,0.796692,0.774638,0.747071,0.717436,0.694693,...,0.361819,0.363198,0.359063,0.353549,0.345968,0.343901,0.345279,0.345279,0.339766,0.353549
27630,0.703353,0.023680,0.021124,0.0,0.543763,0.603722,0.589938,0.564438,0.535493,0.511371,...,0.175052,0.175052,0.173673,0.161957,0.161957,0.164025,0.160579,0.151620,0.146795,0.141282
13438,0.516617,0.349011,0.731921,0.0,0.002068,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
19696,0.582633,0.302842,0.718413,0.0,0.002068,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [6]:
train_inputs.shape[0]

16000

In [7]:
torch.manual_seed(1)

In [8]:
train_data_length = train_inputs.shape[0]
train_data = torch.from_numpy(train_data_pd.values).float().to('cpu')
train_labels = torch.zeros(train_data_length)
train_set = [
    (train_data[i], train_labels[i]) for i in range(train_data_length)
]
train_data.shape

torch.Size([16000, 255])

In [9]:
from torch.utils.data import Dataset

class ConditionalTimeSeriesDataset(Dataset):
    def __init__(self, time_series_data, condition_data):
        """
        time_series_data: Tensor of shape (N, 252)
        condition_data: Tensor of shape (N, 3)
        """
        self.time_series_data = time_series_data
        self.condition_data = condition_data

    def __len__(self):
        return len(self.time_series_data)

    def __getitem__(self, idx):
        return self.time_series_data[idx], self.condition_data[idx]

In [10]:
from torch.utils.data import DataLoader

train_dataset = ConditionalTimeSeriesDataset(train_outputs, train_inputs)
valid_dataset = ConditionalTimeSeriesDataset(valid_outputs, valid_inputs)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)

In [11]:
class ConditionalDiscriminator(nn.Module):
    def __init__(self, input_dim=252, condition_dim=3):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim + condition_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x, conditions):
        # Concatenate time series and condition
        x = torch.cat([x, conditions], dim=1)
        return self.model(x)

In [12]:
discriminator = ConditionalDiscriminator()

In [13]:
class ConditionalGenerator(nn.Module):
    def __init__(self, latent_dim=100, condition_dim=3, output_len=252, hidden_dim=64):
        super().__init__()
        self.input_proj = nn.Linear(latent_dim + condition_dim, hidden_dim)
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.output_proj = nn.Linear(hidden_dim, 1)
        self.output_len = output_len

    def forward(self, z, conditions):
        batch_size = z.size(0)
        x = torch.cat([z, conditions], dim=1)
        x = self.input_proj(x)  # (B, H)

        # Repeat across time steps
        x = x.unsqueeze(1).repeat(1, self.output_len, 1)  # (B, T, H)
        lstm_out, _ = self.lstm(x)
        out = self.output_proj(lstm_out).squeeze(-1)  # (B, T)
        return out

generator = ConditionalGenerator()

In [14]:
lr = 0.001
num_epochs = 300
loss_function = nn.BCELoss()

In [15]:
optimizer_discriminator = torch.optim.Adam(discriminator.parameters(), lr=lr)
optimizer_generator = torch.optim.Adam(generator.parameters(), lr=lr)

In [ ]:
losses_discriminator = []
losses_generator = []
steps = []

for epoch in range(num_epochs):
    for n, (real_samples, condition_vectors) in enumerate(train_loader):
        # real_samples: (batch_size, 252)
        # condition_vectors: (batch_size, 3)

        real_samples_labels = torch.ones((batch_size, 1))
        generated_samples_labels = torch.zeros((batch_size, 1))

        # Sample latent space
        latent_dim = 100
        latent_space_samples = torch.randn((batch_size, latent_dim))  # Latent space dimension

        # Ensure condition_vectors has the same batch size as latent_space_samples
        condition_vectors = condition_vectors.expand(batch_size, -1)  # Repeat condition vectors to match batch size

        # === Generator step ===
        # Pass latent_space_samples and condition_vectors separately to the generator
        generated_samples = generator(latent_space_samples, condition_vectors)

        # === Prepare discriminator input ===
        all_samples = torch.cat((real_samples, generated_samples.detach()), dim=0)
        all_conditions = torch.cat((condition_vectors, condition_vectors), dim=0)  # Duplicate for both real and generated
        all_labels = torch.cat((real_samples_labels, generated_samples_labels), dim=0)

        # === Train discriminator ===
        discriminator.zero_grad()
        output_discriminator = discriminator(all_samples, all_conditions)
        loss_discriminator = loss_function(output_discriminator, all_labels)
        loss_discriminator.backward()
        optimizer_discriminator.step()

        # === Train generator ===
        generator.zero_grad()
        # Generate again to avoid using detached samples
        generated_samples = generator(latent_space_samples, condition_vectors)
        output_discriminator_generated = discriminator(generated_samples, condition_vectors)
        loss_generator = loss_function(output_discriminator_generated, real_samples_labels)
        loss_generator.backward()
        optimizer_generator.step()

        # === Logging (every 100 iterations per epoch) ===
        if n % 100 == 0 and epoch % 25 == 0:  # Adjust this number to control how often you log
            losses_discriminator.append(loss_discriminator.item())
            losses_generator.append(loss_generator.item())
            steps.append(epoch * len(train_loader) + n)
            print(f"Epoch: {epoch} Step: {n} Loss D.: {loss_discriminator.item()}")
            print(f"Epoch: {epoch} Step: {n} Loss G.: {loss_generator.item()}")


Epoch: 0 Step: 0 Loss D.: 0.6944021582603455
Epoch: 0 Step: 0 Loss G.: 0.7369272708892822
Epoch: 0 Step: 100 Loss D.: 0.2651016116142273
Epoch: 0 Step: 100 Loss G.: 1.610642910003662
Epoch: 0 Step: 200 Loss D.: 0.21237897872924805
Epoch: 0 Step: 200 Loss G.: 2.838747262954712
Epoch: 0 Step: 300 Loss D.: 0.3313138484954834
Epoch: 0 Step: 300 Loss G.: 2.0783228874206543
Epoch: 0 Step: 400 Loss D.: 0.3305582106113434
Epoch: 0 Step: 400 Loss G.: 2.8780431747436523
Epoch: 25 Step: 0 Loss D.: 0.46152371168136597
Epoch: 25 Step: 0 Loss G.: 1.3781428337097168
Epoch: 25 Step: 100 Loss D.: 0.6416344046592712
Epoch: 25 Step: 100 Loss G.: 1.0354175567626953
Epoch: 25 Step: 200 Loss D.: 0.5932410955429077
Epoch: 25 Step: 200 Loss G.: 1.1954452991485596
Epoch: 25 Step: 300 Loss D.: 0.5336039066314697
Epoch: 25 Step: 300 Loss G.: 1.112735390663147
Epoch: 25 Step: 400 Loss D.: 0.5231232643127441
Epoch: 25 Step: 400 Loss G.: 1.1205263137817383
Epoch: 50 Step: 0 Loss D.: 0.587220311164856
Epoch: 50 Step

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(steps, losses_discriminator, label='Discriminator Loss')
plt.plot(steps, losses_generator, label='Generator Loss')
plt.xlabel('Training Steps')
plt.ylabel('Loss')
plt.title('Discriminator and Generator Loss During Training')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
torch.save(generator.state_dict(), "generator_gan_ecv_lstm.pth")
torch.save(discriminator.state_dict(), "discriminator_gan_ecv_lstm.pth")

In [ ]:
import random

# Pick N random examples from your validation set
n_samples = 100
indices = random.sample(range(len(valid_dataset)), n_samples)

real_samples = []
real_conditions = []

for idx in indices:
    real_ts, condition = valid_dataset[idx]
    real_samples.append(real_ts)
    real_conditions.append(condition)

real_samples = torch.stack(real_samples)           # (n_samples, 252)
real_conditions = torch.stack(real_conditions)     # (n_samples, 3)

In [ ]:
generator.eval()
with torch.no_grad():
    latent_dim = 100  # or whatever your latent size is
    z = torch.randn(n_samples, latent_dim)
    generated_samples = generator(z, real_conditions)  # (n_samples, 252)

In [ ]:
import matplotlib.pyplot as plt

# Calculate the global min and max for both real and generated samples
min_val = min(real_samples.min(), generated_samples.min())
max_val = max(real_samples.max(), generated_samples.max())

# Assuming real_samples, generated_samples, and real_conditions are already in the correct shape

for i in range(n_samples):
    plt.figure(figsize=(10, 3))
    plt.plot(real_samples[i].cpu().numpy(), label="Real", linewidth=2)
    plt.plot(generated_samples[i].cpu().numpy(), label="Generated", linestyle="--")
    
    # Set the axis limits based on global min and max
    plt.xlim(0, len(real_samples[i]))  # Standardize x-axis (if you know your x range)
    plt.ylim(min_val, max_val)  # Standardize y-axis

    plt.title(f"Comparison for Condition: {real_conditions[i].cpu().numpy()}")
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
def evaluate_model_gan(real_samples, generated_samples):

    # convert list to numpy arrays
    predicted_values = generated_samples.cpu().numpy()  # [N, 255, 1]
    true_values = real_samples.cpu().numpy()            # [N, 255, 1]

    # flatten for metrics
    y_pred_flat = predicted_values.flatten()
    y_true_flat = true_values.flatten()
    
    # compute evaluation metrics
    mse = mean_squared_error(y_true_flat, y_pred_flat)
    mae = mean_absolute_error(y_true_flat, y_pred_flat)
    r2 = r2_score(y_true_flat, y_pred_flat)

    print(f"Validation MSE: {mse:.6f}")
    print(f"Validation MAE: {mae:.6f}")
    print(f"Validation R² Score: {r2:.6f}")

    return mse, mae, r2

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
evaluate_model_gan(real_samples, generated_samples)